In [2]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.ui.showConsoleProgress", "true") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/20 20:46:16 WARN Utils: Your hostname, Siddhants-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.5 instead (on interface en0)
25/11/20 20:46:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/20 20:46:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/20 20:46:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/20 20:46:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
path = "/Users/siddhantgupta/Desktop/Code/Practice Set/Spark/src/data/onlinedeliverydata.csv"
df = spark.read.csv(path, header=True, inferSchema=True)  
df.createOrReplaceTempView("view")
df.show()

+---+------+--------------+--------------+---------------+--------------------------+-----------+--------+---------+--------+------------------+------------+---------+--------+--------------------+--------------------+-------------------+--------------+-----------------------+-------------------+------------------------+-----------------+--------------------+--------------+--------------+-----------------+--------------+-------------------+--------------+-----------------+------------------+-----------------------------------------+----------------------------------------+---------------------+-----------------+-----------------------+-----------------+-------------------+-----------------+--------------------------+--------------------+-------------------+---------------------+-----------------------+-------------------+--------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------

In [6]:
df_no_nulls = df.na.drop()
df_no_nulls.show()

+---+------+--------------+--------------+---------------+--------------------------+-----------+--------+---------+--------+------------------+------------+---------+--------+--------------------+--------------------+-------------------+--------------+-----------------------+-------------------+------------------------+-----------------+--------------------+--------------+-----------------+-----------------+-----------------+-------------------+-----------------+-----------------+------------------+-----------------------------------------+----------------------------------------+---------------------+-----------------+-----------------------+-----------------+-------------------+-----------------+--------------------------+--------------------+-------------------+---------------------+-----------------------+-------------------+--------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------

In [7]:
# get distinct values (and counts) for every column in df_no_nulls
# Results saved in `distinct_summary`. Prints up to 100 distinct values per column.
distinct_summary = {}

for col in df_no_nulls.columns:
    cnt = df_no_nulls.select(F.col(col)).distinct().count()
    vals = df_no_nulls.select(F.col(col)).distinct().limit(100).rdd.flatMap(lambda x: x).collect()
    distinct_summary[col] = {"distinct_count": cnt, "sample_values": vals}
    print(f"Column: {col}  - Distinct count: {cnt}")
    print("Values (up to 100):", vals)
    print("-" * 80)

# `distinct_summary` can be inspected programmatically, e.g.:
# distinct_summary['Age']['distinct_count']
# distinct_summary['Age']['sample_values']

Column: Age  - Distinct count: 16
Values (up to 100): ['29', '30', '22', '28', '31', '18', '27', '26', '19', '23', '25', '33', '24', '32', '20', '21']
--------------------------------------------------------------------------------
Column: Gender  - Distinct count: 2
Values (up to 100): ['Female', 'Male']
--------------------------------------------------------------------------------
Column: Marital Status  - Distinct count: 3
Values (up to 100): ['Prefer not to say', 'Married', 'Single']
--------------------------------------------------------------------------------
Column: Occupation  - Distinct count: 4
Values (up to 100): ['Student', 'House wife', 'Employee', 'Self Employeed']
--------------------------------------------------------------------------------
Column: Monthly Income  - Distinct count: 5
Values (up to 100): ['No Income', '10001 to 25000', '25001 to 50000', 'More than 50000', 'Below Rs.10000']
----------------------------------------------------------------------------